# gopigo3 model

The final objective of this course will be Autonomous navigation and SLAM(Simultaneous Localization and Mapping).
Autonomous navigation refers that the robot is able to move autonomously around the environment avoiding any obstacle.

In a hospital, a delivery robot carries samples or food from one room to another. 

The main objectives are:
- Create a real robot and its virtual model for simulation purposes
- locate the robot in our hospital environment
- perform autonomous navigation in the Hospital environment
- generate and store a map of the hospital 
- use SLAM techniques to find an optimal trajectory to reach a speciffic hospital target position

To fulfill these objectives, use the VS Code and clone the repository:

https://github.com/manelpuig/rUBotCoop_LabProject


<img src="./Images/1_gopigo.png">

First of all, open the .bashrc file and verify the ROS_IP and ROS_MASTER_URI environment variables and source to the proper workspace:

source ~/gopigo_pc_ws/devel/setup.bash

export ROS_IP=192.168.18.84

export ROS_MASTER_URI=http://192.168.18.84:11311

Let's now review the different objectives:

### rUBotCoop model generation

To perform autonomous Navigation, different sensors are needed and installed in the robot:
- a two-dimensional camera: correspondas to RBPi camera
- a Laser Distance Sensor (LDS): is the unidirectional distance sensor of the GoPiGo3 kit
- a 360º LIDAR sensor: corresponds to the EAI YDLIDAR X4 (https://www.robotshop.com/es/es/escaner-laser-360-ydlidar-x4.html)

We have created a speciffic ROS Package to define a complete model of gopigo robot including these sensors. This package is called "gopigo3_description" and the robot model is saved in URDF folder as "gopigo3.gazebo" file. This URDF model contains the SDF (Gazebo tags) for a complete, dynamic simulation. This package provides the gopigo3_rviz.launch launch file to interactively visualize the model in RViz.

Launch the ROS visualization tool to check that the model is properly built. This tool is RViz and only represents the robot visual features. You have available all the options to check every aspect of the appearance of the model:

In [ ]:
roslaunch gopigo3_description gopigo3_rviz.launch

<img src="./Images/LP01_full_model_rviz.png">

Some modifications in the initial model "gopigo3_init.gazebo" have been made to:
- locate the LIDAR in the correct position
- locate the camera axis in front position to see properly

You have to identify the frames positions (rpy="0 0 3.14) and gazebo pluggins:

In [ ]:
<!-- Laser Distance Sensor JOINT base_link -->
  <joint name="scan_joint" type="fixed">
    <parent link="base_link"/>
    <child link="base_scan"/>
    <origin xyz="0 0 0.125" rpy="0 0 3.14"/>
  </joint>

In [ ]:
<!-- Laser Distance Sensor YDLIDAR X4 controller-->
  <gazebo reference="base_scan">
    <sensor type="ray" name="lds_lfcd_sensor">
      <pose>0 0 0 3.14 0 0</pose>

Special attention has to be done to the YDLIDAR sensor.

- YDLIDAR X4 is a 360-degree two-dimensional rangefinder (hereinafter referred to as X4) developed by YDLIDAR team. 
- Based on the principle of triangulation, it is equipped with related optics, electronics and algorithm design to achieve high-frequency and high-precision distance measurement. 
- The mechanical structure rotates 360 degrees to continuously output the angle information as well as the point cloud data of the scanning environment while ranging.
-10m Ranging distance (2cm absolute error)

<img src="./Images/LP01_ydlidar.png">

### rUBotCoop in world environment

In robotics research, always before working with a real robot, we simulate the robot behaviour in a virtual environment close to the real one. The dynamic simulation of a robot, is a better approach to examining the actual behavior of the robot rather than just using software. Rigid body mechanics, including mass and inertia, friction, damping, motor controllers, sensor detection properties, noise signals, and every aspect of the robot and the environment that can be retained in a model with reasonable accuracy is much less expensive when replicated in a simulator than if you tried to do this with physical hardware.

Gazebo is an open source 3D robotics simulator and includes an ODE physics engine and OpenGL rendering, and supports code integration for closed-loop control in robot drives. This is sensor simulation and actuator control.

A speciffic ROS Package called "gazebo_control" have been created for this purpose.

In [ ]:
roslaunch gazebo_control spawn.launch
roslaunch gopigo3_description gopigo3_rviz.launch

<img src="./Images/LP02_rubotcoop_spawn.png">

<img src="./Images/LP02_rubotcoop_spawn_rviz.png">

<img src="./Images/LP05_rqt_gopigo.png">

In order to kill the previous Gazebo process, type:

killall gzserver && killall gzclient

### Design the Project world

Here we have first to design the project world, for exemple a hospital floor with different rooms where our rUBotCoop delivery robot has to navigate autonomously.

There is a very useful and simple tool to design a proper world: Building editor" in gazebo.

Open gazebo as superuser:

In [ ]:
sudo gazebo

You can build your world using "Building Editor" i Edit menu

<img src="./Images/LP03_BuildingWorld1_1.png">

You can save:
- the generated model in a model folder
- the generated world (world1.world) in the world folder.


### Exercise:
Generate a proper world with different rooms 

### Spawn the gopigo robot

Now, spawn the rUBotCoop robot in our generated world. You have to create a "rubot_navigation.launch" file:

In [ ]:
<launch>
  <arg name="world" default="world1.world"/> 
  <arg name="model" default="gopigo3.gazebo" />
  <arg name="x_pos" default="0.0"/>
  <arg name="y_pos" default="0.0"/>
  <arg name="z_pos" default="0.0"/>

  <include file="$(find gazebo_ros)/launch/empty_world.launch">
    <arg name="world_name" value="$(find gazebo_control)/worlds/$(arg world)"/>
    <arg name="paused" value="false"/>
    <arg name="use_sim_time" value="true"/>
    <arg name="gui" value="true"/>
    <arg name="headless" value="false"/>
    <arg name="debug" value="false"/>
  </include>

  <param name="robot_description" textfile="$(find gopigo3_description)/urdf/$(arg model)" />
  
  <node pkg="gazebo_ros" type="spawn_model" name="spawn_urdf"
    args="-urdf -model gopigo3 -x $(arg x_pos) -y $(arg y_pos) -z $(arg z_pos) -param robot_description" />

</launch>

In [ ]:
roslaunch gazebo_control rubot_navigation.launch

<img src="./Images/LP04_BuildingWorld1_2.png">

To control the robot with the Keyboard you type:

In [ ]:
rosrun key_teleop key_teleop.py /key_vel:=/cmd_vel

In order to kill the previous Gazebo process, type:

killall gzserver && killall gzclient

or type ctrl+r and k

### ROS autonomous navigation in the new world environment

Once the world has been generated we will create a ROS Package "rubot_control" to perform the autonomous navigation 

<img src="./Images/LP07_rubot_control.png">

In [ ]:
cd ~/rUBotCoop_LabProject/src
catkin_create_pkg rubot_control rospy std_msgs sensor_msgs geometry_msgs nav_msgs
cd ..
catkin_make

We will create now different navigation python files in "src" folder:
- rubot_autonomous_navigation1.py
- rubot_autonomous_navigation2.py

we will create also a "launch" folder including the corresponding launch files:
- rubot_nav1.launch
- rubot_nav2.launch

Test the autonomous navigation obtained using the python file typing:

In [ ]:
roslaunch rubot_control rubot_nav2.launch

Careful:
- we have included in launch file: gazebo spawn, rviz visualization and node in navigation python file
- In rviz you have to change the fixed frame to "odom" frame

To control the robot with the Keyboard you type:

In [ ]:
rosrun key_teleop key_teleop.py /key_vel:=/cmd_vel

<img src="./Images/LP08_rubot_nav_key.png">

To perform the Autonomous Navigation we have designed different python files:

- "rubot_autonomous_navigation1.py": The Python script makes the robot wander in the environment while avoiding the obstacles. To do this, we have implemented the following rules in our script: 
    - If there is no obstacle, move forward at a reference speed of 0.8 m/s. 
    - If the range provided by the distance sensor is lower than 2 meters, go back and rotate counter-clockwise until avoiding the obstacle. 
    - Since the distance sensor throws unidirectional measurements, we should check the measurements from the LDS to find if there are obstacles to the sides, and the threshold should be lower than 1.6 meters. If obstacles are detected, go back and rotate counter-clockwise faster to avoid the obstacle and not get stuck on it.

<img src="./Images/01_rubot_nav_rqt.png">

### Exercise:

Create another rubot_autonomous_navigation.py file to improve the navigation process of our gopigo3 rubot